# Notebook 1) Skyrmion U-Net training & first prediction tutorial

This notebook demonstrates how to define a neural network using a mini U-Net model, as well as how to train it, make predictions, and benchmark its performance. All the methods required throughout this workflow are introduced. 

For training the mini U-Net model, a small dataset is used, making it feasible to train on a CPU within a reasonable timeframe. After that, the process of training a larger U-Net will be shown; the approach remains the same, with only certain parameters increased.

## 0. Configure notebook 

In [ ]:
try:
    import google.colab
    in_colab = True
    ![ ! -d "AI-Magnetism-Session-Regensburg-2025" ] ||  [ ! -d "AI-Magnetism-Session-Regensburg-2025/.git" ] && git clone https://github.com/kfjml/AI-Magnetism-Session-Regensburg-2025
    ! pip install "numba>=0.61.0,<0.62" "tensorflow[and-cuda]>=2.16.2,<3" "albumentations>=2.0.4,<3"  "pandas>=2.2.2,<3" "chardet>=5.2.0,<6" "opencv-python-headless>=4.11.0.86,<5" "wget>=3.2,<4" "pyyaml>=6.0.2, <7" "pillow>=11.1.0, <12"
    ! pip install "ipympl>=0.9.6" "ipywidgets>=7.7.1" "matplotlib>=3.10.0,<4"
    basis_dir = "/content/AI-Magnetism-Session-Regensburg-2025/"
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    basis_dir = "./"
    in_colab = False

mini_unet_loaded_from_repository = False

### If you are running this notebook in **Google Colab**, after executing the first cell (cell above), go to **Runtime → Restart session**, then rerun the first cell. After that, you can execute the cells below. This is necessary because some required packages are installed in Google Colab and need a restart to take effect.

## 1. Import packages

In this notebook, the key packages used are **TensorFlow** and **Keras**, essential for building, training, and performing inference with deep learning models efficiently. TensorFlow provides a robust platform for machine learning, while Keras simplifies model development with its user-friendly API. Additional packages are included to create an ecosystem that enables both training and inference processes. Additionally, the following packages are used:
- `albumentations` - Image augmentation library for computer vision.  
- `cv2 (OpenCV)` - Used here only in combination with `albumentations`.  
- `PIL (Pillow)` - Image processing library.  
- `numpy` - Numerical computing library for tensors.  
- `matplotlib.pyplot` - Visualization library for plotting.  
- `pandas` - Data (table) manipulation and analysis library.  
- `glob` - File path management tool.  
- `os` - Interface for file/folder creation, existence checks, filename handling, and more.  
- `time` - Used to measure execution time, among other functions.  

In [ ]:
import tensorflow as tf
import albumentations
import cv2

from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import time

### 1.1. Check if GPU is available

If a GPU is available, we use the float16 policy for better performance, as it allows for faster computation and reduced memory usage.

In [ ]:
#function that returns true if a GPU is available
gpu_available = lambda : len(tf.config.list_physical_devices('GPU'))

#in the case of GPU, switch to mixed_float16 policy
if gpu_available():
    print("GPU is available")
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

## 1.2 Load Dataset Table

Load the dataset table containing the **Kerr micrograph images** and **segmentation mask labels**, which are the ground truth. A **segmentation mask** is an image where each pixel is assigned a class label, represented using different colors, to highlight distinct regions and specify how each pixel should be classified into a particular class.

In machine learning, **ground truth** is the benchmark used to evaluate or train a model. 

Additionally, the same **source ID** indicates that the images originate from the same video.

In [ ]:
dataset_table = pd.read_csv(basis_dir+"dataset/table.csv",sep=";")
dataset_table["img_fn"] = dataset_table.apply(lambda x:basis_dir+x["img_fn"],axis=1)
dataset_table["label_fn"] = dataset_table.apply(lambda x:basis_dir+x["label_fn"],axis=1)
dataset_table

Load of an example of this dataset. The first dimension/axis of the array is the height, the second is the width, and in the case of the Kerr micrographs, the third is the RGB channel. In the case of grayscale images (the labels are color-coded, while the Kerr micrographs are black and white), there are no multiple channels, so there is no fourth axis.

In [ ]:
ix = 110
kerr_micrograph = np.array(Image.open(dataset_table.loc[ix]["img_fn"]))
print("kerr_micrograph\n",kerr_micrograph)
segmentation_mask_label = np.array(Image.open(dataset_table.loc[ix]["label_fn"]))
print("segmentation_mask_label\n",segmentation_mask_label)

Plot of this example

In [ ]:
fig,ax = plt.subplots(ncols=2,dpi=300,figsize=(10,10))
ax[0].imshow(kerr_micrograph,cmap="grey")
ax[0].set_title("Kerr micrograph")
ax[1].imshow(segmentation_mask_label)
ax[1].set_title("Groundtruth\n"+"Segmentation mask label")
ax[0].set_xlabel("x [px]")
ax[1].set_xlabel("x [px]")
ax[0].set_ylabel("y [px]")
ax[1].set_ylabel("y [px]")

With the segmentation mask for each Kerr microscopy image, it becomes easy to identify the objects (such as skyrmions) and determine their geometric properties, providing a basis for future analysis. That's why we aim to **train** this segmentation mask.

## 1.3 Segmentation Mask Label

The Kerr micrographs are labeled with a segmentation mask. The segmentation mask in the dataset consists of five distinct classes:

- **Skyrmions** — RGB label: red [255, 0, 0]  
- **Defects** — RGB label: green [0, 255, 0]  
- **Ferromagnetic (FM) background** — RGB label: blue [0, 0, 255]  
- **Non-Ferromagnetic (FM) background** — RGB label: yellow [255, 255, 0]  
- **Boundary non-Ferromagnetic/Ferromagnetic background** — RGB label: cyan [0, 255, 255]  


In [ ]:
text_labels = [("Skyrmions",(1,0,0)),("Defects",(0,1,0)),("Ferromagnetic (FM) background",(0,0,1)),
               ("Non-Ferromagnetic (FM) background",(1,1,0)),("Boundary non-Ferromagnetic/Ferromagnetic background",(0,1,1))]
fig,ax = plt.subplots(figsize=(5, 2),dpi=130,facecolor="k")
for i, (label, color) in enumerate(text_labels):
    ax.text(0.5, 1 - i * 0.2, label, fontsize=15, color=color, ha='center')
ax.set_ylim(0,1.2)
ax.axis("off")
ax.set_title("U-Net classes",fontsize=20,color="white")

In [ ]:
fig,ax = plt.subplots(ncols=2,dpi=300,figsize=(12,3.2))
ax[0].imshow(kerr_micrograph,cmap="grey")
ax[0].set_title("Kerr micrograph")
ax[1].imshow(segmentation_mask_label)
ax[1].set_title("Groundtruth\n"+"Segmentation mask label")
ax[0].set_xlabel("x [px]")
ax[1].set_xlabel("x [px]")
ax[0].set_ylabel("y [px]")
ax[1].set_ylabel("y [px]")

cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3,4,5],6), 
                    cmap=matplotlib.colors.ListedColormap([(0, 1, 1), (1, 1, 0), (0, 0, 1), (0, 1, 0), (1, 0, 0)])),
                    ax=ax[1], ticks=[0.5, 1.5, 2.5,3.5,4.5])
cbar.set_ticklabels(['FM-non FM Boundary', 'non FM Bakckground', 'FM Bakckground', 'Defects', 'Skyrmions'])

We don't want to train the 5 classes, but we aim to train a 3 classes. Furthermore, instead of the RGB labels, the classes are assigned a class index. 

The following classes are used:

- **Skyrmions** - Index 0, RGB label: red [255, 0, 0]  
- **Defects** - Index 1, RGB label: green [0, 255, 0]
- **Background** - Index 2, RGB label: blue [0, 0, 255]  
   - The background class includes:
     - The ferromagnetic (FM) background  
     - The non-ferromagnetic (non-FM) background  
     - The boundary between ferromagnetic and non-ferromagnetic backgrounds

Let's now define the conversion function from RGB index to class index and vice versa.

In [ ]:
def trafo_rgb_to_class(I):
    Q = np.zeros((I.shape[0],I.shape[1]),dtype=np.uint8)
    R,G,B = I[:,:,0],I[:,:,1],I[:,:,2]
    skyrmion_mask = (R>=128)&(G<128)&(B<128)
    defect_mask = (R<128)&(G>=128)&(B<128)
    bck_mask = ~(skyrmion_mask|defect_mask)
    Q[skyrmion_mask] = 0
    Q[defect_mask] = 1
    Q[bck_mask] = 2
    return Q

def trafo_class_to_rgb(I):
    basis = np.array([[255,0,0],[0,255,0],[0,0,255],[255,255,0],[0,255,255]],dtype=np.uint8)
    return basis[I]

In [ ]:
print("segmentation_mask_label.shape",segmentation_mask_label.shape)
print("segmentation_mask_label=\n",segmentation_mask_label)

In [ ]:
segmentation_mask_label_class = trafo_rgb_to_class(segmentation_mask_label)
print("segmentation_mask_label_class.shape",segmentation_mask_label_class.shape)
print("segmentation_mask_label_class=\n",segmentation_mask_label_class)

In [ ]:
fig,ax = plt.subplots(ncols=3,dpi=300,figsize=(10.4,2.4),constrained_layout=True)
ax[0].imshow(kerr_micrograph,cmap="gray")
ax[2].imshow(trafo_class_to_rgb(trafo_rgb_to_class(segmentation_mask_label)))
im = ax[1].imshow(segmentation_mask_label_class,cmap="gray")
col = fig.colorbar(im,ax=ax[1],label="class")
col.set_ticks([0,1,2])
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                    cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                    ax=ax[2], ticks=[0.5, 1.5, 2.5])
cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])
ax[0].set_title("Kerr micrograph")
ax[2].set_title("Groundtruth\n"+"Mask (RGB code for the 3 classes)")
ax[1].set_title("Groundtruth\n"+"Mask (Class index for the 3 classes)")

## 2. Define U-Net architecture

We now want to build this U-Net architecture step by step while understanding the different components involved.

In [ ]:
#define plot function for figures
def plotfig(fn,dpi):
    fig,ax = plt.subplots(dpi=dpi)
    ax.imshow(plt.imread(fn))
    ax.axis("off")
    
plotfig(basis_dir+"notebook_figures/u_net_architecture_2.png",320)

### 2.1. U-Net Layers

First, we define some plotting functions:

In [ ]:
def PlotArray(ax,A,plot_value=True):
    im = ax.imshow(A,cmap="gray",origin="lower")
    fig.colorbar(im,ax=ax)
    ax.set_ylabel("2. axis/dimension\n (y-axis of the image)")
    ax.set_xlabel("3. axis/dimension\n (x-axis of the image)")
    if plot_value:
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                ax.text(j,i, f'{A[i, j]:.2f}', ha='center', va='center',
            color='white' if (A[i, j]-np.min(A))/(np.max(A)-np.min(A)) < 0.5 else 'black')

    ax.set_xticks(np.arange(A.shape[1]))
    ax.set_yticks(np.arange(A.shape[0]))

#### 2.1.1 Tensors

We will use tensors in the following. Tensors (for this workshop) are multi-dimensional arrays that represent data in various shapes and types.

For our tensors $T_{i,x,y,c}$, the first dimension/axis represents the image index (size: batch size), the second corresponds to the y-axis (size: height), the third to the x-axis (size: width), and the fourth represents the channels per pixel (size: number of channels per pixel). For example, each channel may represent the probability that a pixel belongs to a specific class.

An example with 4 images of size 5x4, each with 4 channels, containing values generated from Gaussian random numbers. Then look at the 3rd channel of the 2nd image.

In [ ]:
A = np.random.randn(4,5,4,4)
fig,ax = plt.subplots(dpi=100)
PlotArray(ax,A[1,:,:,2])
ax.set_title("1st axis = 1 (second image),\n4th axis = 2 (third channel)")

A tensor is also the set of Kerr micrograph images stored in our repository. These can also be represented as a tensor with the shape (number of Skyrmion images, width, height, channel index). Additionally, we normalize the data between 0 and 1 for training and prediction. Normalization is very important for inputs and the layers in between.

In [ ]:
print("kerr_micrograph.shape",kerr_micrograph.shape)
kerr_micrograph_tensor = kerr_micrograph[None,:,:,None]/255
print("kerr_micrograph_tensor.shape",kerr_micrograph_tensor.shape)
kerr_micrograph_tensor

In [ ]:
fig,ax = plt.subplots(dpi=200)
im = ax.imshow(kerr_micrograph_tensor[0,:,:,0],cmap="gray")
fig.colorbar(im)
ax.set_ylabel("2. axis/dimension")
ax.set_xlabel("3. axis/dimension")

#### 2.1.2 Convolutional layer

In [ ]:
plotfig(basis_dir+"notebook_figures/convolution_layer.png",120)

In [ ]:
plotfig(basis_dir+"notebook_figures/conv_1.png",300)
plotfig(basis_dir+"notebook_figures/conv_2.png",300)
plotfig(basis_dir+"notebook_figures/conv_3.png",300)

An important layer that does have weights is the **Convolution Layer**, which performs a convolution — a linear operation. The weights in this layer are learned during training. The convolutional layer implemented in TensorFlow using `Conv2D` consists of a convolution and a bias layer. The kernel is a small tensor that acts as a filter with defined dimensions, moving across the input and computing dot products with overlapping regions, effectively performing the convolution. Notably, TensorFlow's `Conv2D` automatically flips the kernel by 180 degrees (both horizontally and vertically) before performing the convolution, ensuring it matches the mathematical definition of convolution. A bias term is then added to the result of each dot product. The kernel size defines the dimensions of this filter, determining the region of the input considered for each output value. With `padding='same'`, zero-padding is added around the input so that the output size matches the input size when the stride is 1. The number of padding pixels added depends on the kernel size and input dimensions.

Convolution can be written using the following formula:  

$$B_{i,x,y,c} = C_c + \sum_{m,n,h} \mathcal{K}_{m,n,h,c} A_{i,x-m,y-n,h}$$

where $ A $ is the input tensor and $ B $ is the output tensor, both of which are 4-dimensional. The $ \mathcal{K} $ denotes the kernel, and $ C_c $ is a channel-specific offset (or bias) that is added. .

Let's now create an example where we set the weights. The first two axes describe the kernel along the x and y axes of the image, while the third and fourth axes are for treatment with multiple channels.

In [ ]:
ConvolutionalLayer = tf.keras.layers.Conv2D(2,kernel_size=(3,3),padding="same")
# initialize convolutional layer size with dummy input
ConvolutionalLayer(np.random.randn(1,6,6,4))
ConvolutionalLayer.get_weights()[1].shape

In [ ]:
ConvolutionalLayer = tf.keras.layers.Conv2D(1,kernel_size=(3,3),padding="same")
# initialize convolutional layer size with dummy input
ConvolutionalLayer(np.random.randn(1,6,6,1))
print("Weights shape:",ConvolutionalLayer.get_weights()[0].shape)
print("Bias shape:",ConvolutionalLayer.get_weights()[1].shape)

kernel = np.array([[[[0]],[[0.5]],[[0]]],
                   [[[1]],[[0]],[[1]]],
                   [[[1]],[[0]],[[0]]]])
bias = np.array([10])

ConvolutionalLayer.set_weights([kernel,bias])

print("Weights:\n",ConvolutionalLayer.get_weights()[0])
print("Bias:\n",ConvolutionalLayer.get_weights()[1])

In [ ]:
fig,ax = plt.subplots(figsize=(7,3.5),ncols=2,dpi=200)
PlotArray(ax[0],kernel[:,:,0,0],False)
PlotArray(ax[1],kernel[::-1,::-1,0,0],False)
ax[0].set_title("Kernel")
ax[0].set_ylabel("1. axis/dimension\n (acts on y-axis of the image)")
ax[0].set_xlabel("2. axis/dimension\n (acts on x-axis of the image)")
ax[1].set_title("Kernel\nhorizontally & vertically\nflipped")
ax[1].set_ylabel("1. axis/dimension\n (acts on y-axis of the image)")
ax[1].set_xlabel("2. axis/dimension\n (acts on x-axis of the image)")
fig.tight_layout()

We use in the following example this kernel, and after the dot product, bias 10 is added.

In [ ]:
A = np.zeros((1,8,8,1))
A[0,2,2] = 10
A[0,5,5] = 10
A[0,2,2] = 10
A[0,1,6] = 10
A[0,1,7] = 10
B = ConvolutionalLayer(A)
fig,ax = plt.subplots(ncols=2,figsize=(6,2.5),dpi=200)
PlotArray(ax[0],A[0,:,:,0],False)
PlotArray(ax[1],B[0,:,:,0],False)
ax[0].set_title("Input")
ax[1].set_title("Output")
fig.tight_layout()

Now, a further example: a larger kernel size with the weights set to a Gaussian distribution.

In [ ]:
A = kerr_micrograph_tensor.copy()
n_kernel = 20
ConvolutionalLayer = tf.keras.layers.Conv2D(1,kernel_size=(n_kernel,n_kernel),padding="same")
# initialize convolutional layer size with dummy input
ConvolutionalLayer(A)

x,y = np.meshgrid(np.arange(n_kernel)-(n_kernel-1)/2,np.arange(n_kernel)-(n_kernel-1)/2)
sigma = 4
K = (np.exp(-(x**2+y**2)/(2*sigma**2)))
K = K.reshape((n_kernel,n_kernel,1,1))
ConvolutionalLayer.set_weights([K,np.array([0.0])])

print("Weights shape:",ConvolutionalLayer.get_weights()[0].shape)
print("Bias shape:",ConvolutionalLayer.get_weights()[1].shape)
B = ConvolutionalLayer(A)
fig,ax = plt.subplots(ncols=2,dpi=300)
ax[0].imshow(A[0,:,:,0],cmap="gray")
ax[1].imshow(B[0,:,:,0],cmap="gray")
ax[0].set_title("Input")
ax[1].set_title("Output")

Now we have an implementation of a Gaussian blur in the form of a tensor transformation.


For the **initialization** of the weights **for training**, we use He Normalization. In this method, the weights are initialized with values drawn from a normal distribution with a mean of zero and a variance that is scaled based on the number of input units. This helps improve convergence and stability.

Now an initialization of the convolutional layer with He normalization:

In [ ]:
A = kerr_micrograph_tensor.copy()/255
ConvolutionalLayer = tf.keras.layers.Conv2D(1,kernel_size=(3,3),padding="same",kernel_initializer="he_normal")
# initialize convolutional layer size with dummy input
ConvolutionalLayer(A)
print("Weights shape:",ConvolutionalLayer.get_weights()[0].shape)
print("Bias shape:",ConvolutionalLayer.get_weights()[1].shape)

B = ConvolutionalLayer(A)
fig,ax = plt.subplots(ncols=2,dpi=300)
ax[0].imshow(A[0,:,:,0],cmap="gray")
ax[1].imshow(B[0,:,:,0],cmap="gray")
ax[0].set_title("Input")
ax[1].set_title("Output")

Here we have now looked at where the input and output are in the channel. In general, what we use in the following applies. In `tf.keras.layers.Conv2D`, the first argument defines the number of output channels, regardless of the number of input channels. Each output channel is generated by applying a separate filter to all input channels. The resulting value at each position is calculated as the sum of the element-wise products between the filter's weights and the corresponding input values across all channels. Consequently, the number of output channels is exactly equal to the value of the first argument. It makes sense to have more output channels than input channels because this way different convolutions can be applied to an image, allowing multiple features or properties to be extracted.

Now an example with multiple channels for input and output.

In [ ]:
ConvolutionalLayer = tf.keras.layers.Conv2D(5,kernel_size=(3,3))
A = np.random.randn(5,20,20,2)
print("Input shape:",A.shape)
print("Output shape:",ConvolutionalLayer(A).shape)
print("Weight/Convolution shape:",ConvolutionalLayer.get_weights()[0].shape)
print("Bias shape:",ConvolutionalLayer.get_weights()[1].shape)

#### 2.1.3 Activation function

In [ ]:
plotfig(basis_dir+"notebook_figures/activation_layer.png",100)

Activation functions are non-linear and crucial for neural networks, enabling them to learn complex patterns and improve performance by introducing non-linear decision boundaries. Without these non-linearities, neural networks would be linear and far less capable of handling complex patterns and data relationships.

In the following we use the Mish function. The Mish function is defined as follows:

$$f(x) = x \cdot \tanh(\log(1 + e^x))$$

The Activation Layer only performs a function and usually does not contain any weight parameters.

In [ ]:
fig,ax = plt.subplots(figsize=(3,3),dpi=100)
x = np.linspace(-10,10)
ax.plot(x,x*np.tanh(np.log(1+np.exp(x))))
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Mish function")

Activation functions are usually combined with e.h. convolution layers; however, here we will focus first solely on the activation function as a standalone layer.

We will now observe what happens when the Mish function is applied to a tensor filled with random numbers. Of course, in practice, these won't be random numbers but rather the data that originates from the input, passes through various layers, and reaches this point in the neural network, where it then passes through the activation function.

In [ ]:
# Basic activation layer
class MishLayer(tf.keras.layers.Layer):
    def call(self, x):
        return tf.keras.activations.mish(x)

#Define a Activation function as a layer
ActivationFunction = MishLayer()
A = np.random.randn(1,4,4,1)*10
B = ActivationFunction(A)

print("A=\n",A)
print("B=\n",B)

fig,ax = plt.subplots(ncols=2,figsize=(7.2,3),dpi=200)
PlotArray(ax[0],A[0,:,:,0])
PlotArray(ax[1],B[0,:,:,0])
ax[0].set_title("Input")
ax[1].set_title("Output")
fig.tight_layout()

#### 2.1.4 Batch normalization

The `tf.keras.layers.BatchNormalization` layer is active only during training. It normalizes the input so that the mean becomes 0 and the standard deviation becomes 1, and then outputs the resulting values.

In [ ]:
BatchNormalization = tf.keras.layers.BatchNormalization(momentum=1e-13)
A = np.random.randn(1,10,10)*10+10
B = BatchNormalization(A,training=True)
print("A=\n",A)
print("B=\n",B)

print("mean A: ",np.mean(A),"std B: ",np.std(A))
print("mean B: ",np.mean(B),"std B: ",np.std(B))

The behavior of batch normalization is more complex than it may seem in this demo. During training, batch normalization does not calculate the mean and standard deviation for each individual input. Instead, it scales and shifts the data using running averages of the mean and standard deviation from previous batches. These moving averages are gradually updated over time.

#### 2.1.5 Convolutional block

In [ ]:
plotfig(basis_dir+"notebook_figures/convolution_layer_w_activation_funtion.png",100)

In [ ]:
# Basic Convolution Block
def conv_block(x, n_channels, param):
    x = tf.keras.layers.Conv2D(n_channels, kernel_size=param["kernel_size"],kernel_initializer=param["kernel_initialization"],padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = MishLayer()(x)
    return x

#### 2.1.5 Double Convolutional Block

In [ ]:
plotfig(basis_dir+"notebook_figures/double_convolution_layer.png",100)

With this we build now a double convolution layer

In [ ]:
def double_conv_block(x, n_channels, param):
    x = conv_block(x,n_channels,param)
    x = conv_block(x,n_channels,param)
    return x

#### 2.1.6 Maximum Pooling

In [ ]:
plotfig(basis_dir+"notebook_figures/max_pooling.png",100)

In [ ]:
plotfig(basis_dir+"notebook_figures/max_pooling_1.png",300)
plotfig(basis_dir+"notebook_figures/max_pooling_2.png",300)
plotfig(basis_dir+"notebook_figures/max_pooling_3.png",300)

The TensorFlow Keras `MaxPooling2D` layer is used to reduce the spatial dimensions (height and width) of an input feature map while retaining important features. It works by applying a pooling operation, such as taking the maximum value from a defined window (e.g., 2x2) as it slides over the input. This process helps reduce the number of parameters, improve computational efficiency, and enhance model robustness by emphasizing the most prominent features in each region. Unlike convolution layers, MaxPooling has no learnable parameters; it simply downsamples the data based on the specified pool size and strides.

Max pooling can be written using the following formula:

$$B_{i, x, y, c} = \max_{m, n} A_{i, s_x x + m, s_y y + n, c}$$

where $ A $ is the input tensor and $ B $ is the output tensor, both of which are 4-dimensional. The terms $ s_x $ and $ s_y $ denote the stride in the horizontal and vertical directions, respectively. The maximum operation is performed over the spatial region defined by the pooling window. Unlike convolution, max pooling does not involve a kernel or additional parameters; it simply selects the maximum value within the defined window.

In [ ]:
MaxPool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
A = np.random.randn(1,4,4,1)
B = MaxPool(A)

print("A=\n",A)
print("B=\n",B)



fig,ax = plt.subplots(ncols=2,figsize=(6,2.5),dpi=200)
PlotArray(ax[0],A[0,:,:,0])
PlotArray(ax[1],B[0,:,:,0])
ax[0].set_title("Input")
ax[1].set_title("Output")
fig.tight_layout()

#### 2.1.7 Dropout Layer

In `tf.keras.layers.Dropout`, a specific dropout rate (first argument) is set, which determines the fraction of input units to be randomly set to zero during training. This technique helps prevent **overfitting** by introducing regularization, making the learning process more robust and improving the model's generalization ability. **Overfitting** occurs when a machine learning model learns the noise and specific details of the training data too well, resulting in poor performance on new, unseen data. This happens because the model becomes too complex, capturing patterns that don't generalize beyond the training set.

In [ ]:
Dropout = tf.keras.layers.Dropout(0.4)
A = np.random.randn(1,5,5,1)
B = Dropout(A,training=True)

print("A=\n",A)
print("B=\n",B)


fig,ax = plt.subplots(ncols=2,figsize=(7.2,3),dpi=200)
PlotArray(ax[0],A[0,:,:,0])
PlotArray(ax[1],B[0,:,:,0])
ax[0].set_title("Input")
ax[1].set_title("Output")
fig.tight_layout()

#### 2.1.8 Downsample block

In [ ]:
plotfig(basis_dir+"notebook_figures/downsampling_block.png",120)

This together gives the downsample block

In [ ]:
def downsample_block(x, n_channels, param):
    f = double_conv_block(x, n_channels, param)
    p = tf.keras.layers.MaxPool2D(pool_size=(2,2))(f)
    p = tf.keras.layers.Dropout(param["dropout"])(p)
    return f, p

#### 2.1.9 Upconvolution Layer

In [ ]:
plotfig(basis_dir+"notebook_figures/up_convolution.png",100)

In [ ]:
plotfig(basis_dir+"notebook_figures/up_conv_1.png",300)
plotfig(basis_dir+"notebook_figures/up_conv_2.png",300)
plotfig(basis_dir+"notebook_figures/up_conv_3.png",300)

Upconvolution is the opposite of max pooling. In upconvolution (also known as transposed convolution or deconvolution), interpolation is performed using a kernel for each pixel. Each pixel (in terms of width and height of an image) is multiplied by a kernel, effectively extrapolating the input to produce a larger output image. This process can be seen as the inverse of convolution: while convolution reduces the spatial dimensions by aggregating information from a larger region into a smaller one, upconvolution expands the spatial dimensions by distributing the value of each input pixel across a larger receptive field through the kernel.


The concept of **strides** is crucial here. Strides determine the step size for moving the kernel across the image. Additionally, **padding** with `"same"` ensures that the final output size matches the original input size.

The final number of output channels is specified in the first argument of the `Conv2DTranspose` function. This value defines the number of output feature maps, and the result is the sum over the input channels after the transposed convolution operation.


The upconvolution (also known as transposed convolution or deconvolution) can be written using the following formula:

$$B_{i, x, y, c} = \sum_{m, n, h} \mathcal{K}_{m, n, h, c} A_{i, \left\lfloor \frac{x}{s_x} \right\rfloor + m, \left\lfloor \frac{y}{s_y} \right\rfloor + n, h} + C_c$$

where $ A $ is the input tensor and $ B $ is the output tensor, both of which are 4-dimensional. The symbol $ \mathcal{K} $ denotes the kernel, and $ C $ is a channel-specific offset (or bias). The terms $ s_x $ and $ s_y $ denote the stride in the horizontal and vertical directions, respectively.   Unlike standard convolution, upconvolution effectively increases the spatial resolution by inserting values in between data points.

In [ ]:
UpConvolutionalLayer = tf.keras.layers.Conv2DTranspose(1,kernel_size=(2,2),strides=2,padding="same")
# initialize convolutional layer size with dummy input
UpConvolutionalLayer(np.random.randn(1,3,3,1))
print("Weights shape:",UpConvolutionalLayer.get_weights()[0].shape)
print("Bias shape:",UpConvolutionalLayer.get_weights()[1].shape)

Kernel = np.array([[[[0]],[[0.5]]],[[[1]],[[0]]]])
Bias = np.array([-3])
UpConvolutionalLayer.set_weights([Kernel,Bias])


A = np.zeros((1,3,3,1))
A[0,0,0] = 1
A[0,1,2] = 0.5
B = UpConvolutionalLayer(A)

print(A)
print(B)


fig,ax = plt.subplots(figsize=(4,3.5),dpi=200)
PlotArray(ax,Kernel[:,:,0,0],False)
ax.set_title("Kernel")
ax.set_ylabel("1. axis/dimension\n (acts on y-axis of the image)")
ax.set_xlabel("2. axis/dimension\n (acts on x-axis of the image)")
fig.tight_layout()

fig,ax = plt.subplots(ncols=2,figsize=(6,2.5),dpi=200)
PlotArray(ax[0],A[0,:,:,0],False)
PlotArray(ax[1],B[0,:,:,0],False)
ax[0].set_title("Input")
ax[1].set_title("Output")
fig.tight_layout()

#### 2.1.10 Concatenation Layer

In [ ]:
plotfig(basis_dir+"notebook_figures/concatenation_layer.png",100)

With the concatenation layer `tf.keras.layers.Concatenate`, you can concatenate tensors - the outputs of different layers - by merging their channels.

In [ ]:
input1 = np.random.randn(1,2,2,2)
input2 = np.random.randn(1,2,2,3)
print("input1=\n",input1)
print("input2=\n",input2)
concat_layer = tf.keras.layers.Concatenate()
output = concat_layer([input1, input2]) 
print("output=\n",output)
print(output.shape)

#### 2.1.11 Upsample block

In [ ]:
plotfig(basis_dir+"notebook_figures/concatenation_2_layer.png",120)
plotfig(basis_dir+"notebook_figures/upsampling_block.png",120)

In [ ]:
def upsample_block(x, conv_features, n_channels, param):
    x = tf.keras.layers.Conv2DTranspose(n_channels*param["upsample_channel_multiplier"], param["kernel_size"], strides=(2,2), padding='same')(x)
    x = tf.keras.layers.concatenate([x, conv_features])
    x = tf.keras.layers.Dropout(param["dropout"])(x)
    x = double_conv_block(x, n_channels, param)
    return x

### 2.2 Tensorflow models

#### 2.2.1 Input layer

In [ ]:
plotfig(basis_dir+"notebook_figures/input_layer.png",120)

The input layer of a model is in tensorflow is defined as

In [ ]:
input_layer = tf.keras.layers.Input(shape=(9,9, 1))

#### 2.2.2 Output layer

In [ ]:
plotfig(basis_dir+"notebook_figures/segmentation_mask_output.png",120)

We will look in the following at the following example's output layer. With the output layer being a convolutional layer with a $ (1, 1) $ kernel size that changes the channel size to 3 from another channel size. After that, the softmax function is applied to each pixel across the channel dimension (indexed by $i$) in the output tensor $\mathbf{z}$, which has dimensions: image height, width, and number of channels. The softmax function converts $\mathbf{z}$ to probabilities:

$$p_{j,x,y,i} = \frac{\exp(\mathbf{z}_{j,x,y,i})}{\sum_{i \in \text{classes}} \exp(\mathbf{z}_{j,x,y,i})}$$

The resulting probability tensor $p$ can then be converted to a class prediction mask $\mathbf{m}_j$ by taking the index of the maximum probability for each pixel:

$$ \mathbf{m}_{j,x,y} = \arg\max_i \; p_{j,x,y,i} $$

This layer can be implemented with `tf.keras.layers.Conv2D(3, (1, 1), padding="same", activation="softmax")`. 

And now an example where the weights of the convolution are set to the identity so that we can now observe the softmax behavior. And now an example where the weights of the convolution are set to the identity matrix so that we can now observe the softmax behavior. Therefore, we also set the input layer channels to 3; otherwise, the convolution layer would adjust them to 3 (as defined for the layer) with the convolution. As an example, we will make the prediction with an image that is one pixel wide and five pixels high, with three channels, to keep it simple. And now we plot the 2nd dimension of the tensor on the y-axis and the **channels** (4th axis, dimension) on the x-axis, not the 3rd dimension/axis.

In [ ]:
output_layer = tf.keras.layers.Conv2D(3, (1,1),padding="same",activation = "softmax")
output_layer(np.random.randn(5,2,2,3))
output_layer.set_weights([np.eye(3)[None,None],np.zeros(3)])

In [ ]:
A = np.random.randn(1,5,1,3)
print("A=\n",A)
B = output_layer(A)
print("A=\n",B)

fig,ax = plt.subplots(ncols=2,figsize=(6,3),dpi=200)
PlotArray(ax[0],A[0,:,0,:])
PlotArray(ax[1],B[0,:,0,:])
ax[0].set_title("Input")
ax[1].set_title("Output")
ax[0].set_ylabel("2. axis/dimension\n (x-axis of the image)")
ax[0].set_xlabel("4. axis/dimension\n (channel axis of the image)")
ax[1].set_ylabel("2. axis/dimension\n (x-axis of the image)")
ax[1].set_xlabel("4. axis/dimension\n (channel axis of the image)")

fig.tight_layout()

#### 2.2.3 Tensorflow model

In TensorFlow, a model is defined as shown in the following example:

In [ ]:
nput_layer = tf.keras.layers.Input(shape=(9, 9, 1))
conv_layer = tf.keras.layers.Conv2D(1, kernel_size=(3, 3), padding="same")(input_layer)
activation_layer = MishLayer()(conv_layer)
output_layer = tf.keras.layers.Conv2D(3, (1,1),padding="same",activation = "softmax")(activation_layer)
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Often in TensorFlow, parameters are referred to as weights, and they are the elements that are trained. These weights of our initialized (untrained) model can be accessed in the following way:

In [ ]:
model.weights

In [ ]:
input_data = np.random.randn(1,9,9, 1).astype(np.float32)
print("input_data=\n",input_data)
output_data = model(input_data)
print("output_data=\n",output_data)

### 2.3. Building the Skyrmion U-Net model/architecture

The U-Net architecture (Ronneberger et al., *LNCS*, Vol. 9351: 234-241 (2015)) generally consists of a:

**Contraction path**  
- Downsampling; acts as a decoder  
- Feature channels double, image size is halved  
- Consists of convolution and max pooling layer
- Usually applied consecutively

**Expansion path**  
- Up-sampling; acts as an encoder  
- Feature channels reduce, image size increases  
- Consists of up-convolution and convolution  
- Concatenation with high-resolution features from the left
- Usually applied consecutively



**Last layer:** 1x1 convolution + softmax  

**Advantage of U-Net:** Few data are necessary for good quality  

We will now consider a "Mini" U-Net architecture. The normal (larger) U-Net architecture is defined further below; everything is the same, only certain parameters are increased. For this larger U-Net, the training would take longer.  

The Mini U-Net is defined as

In [ ]:
plotfig(basis_dir+"notebook_figures/u_net_architecture_2.png",300)

U-Net can be described as follows:

$$ \mathbf{z}_j = \mathbf{f}_{\vec{\theta}}(\mathbf{x}_j) $$

where $\vec{\theta}$ is a high-dimensional vector containing all parameters, $\mathbf{z}_j$ is a 3-dimensional output tensor, and $\mathbf{x}_j$ is the input Kerr microscopy image ($j$ is a index to label different tensors). In $\mathbf{f}$ are encapsulated all the operations such as convolution, max pooling, up-convolution, batch normalization, ... .

$\mathbf{z}_j$ is converted to probabilities using softmax:

$$ p_{j,x,y,i} = \frac{\exp(\mathbf{z}_{j,x,y,i})}{\sum_{i\in \mathrm{classes}} \exp(\mathbf{z}_{j,x,y,i})} $$

The output mask $\mathbf{m}_j$ is then determined by:

$$ \mathbf{m}_{j,x,y} = \mathrm{arg\,max}_i\; p_{j,x,y,i}$$

Now we can define the U-Net architecture using these functions. 

In [ ]:
def get_unet(param):
    input = tf.keras.layers.Input(shape=param["input_shape"]+(1,))
    next_input = input
    
    l_residual_con = []
    for i in range(param["n_depth"]):
        residual_con,next_input = downsample_block(next_input, 
                                                   (2**i)*param["filter_multiplier"],
                                                   param)
        l_residual_con.append(residual_con)

    next_input = double_conv_block(next_input,
                                   (2**param["n_depth"])*param["filter_multiplier"],
                                   param)

    for i in range(param["n_depth"]):
        next_input = upsample_block(next_input,
                                    l_residual_con[param["n_depth"]-1-i],
                                    (2**(param["n_depth"]-1-i))*param["filter_multiplier"],
                                    param)

    output = tf.keras.layers.Conv2D(param["n_class"], 
                                    (1,1),
                                    padding="same",
                                    activation = "softmax",
                                    dtype='float32')(next_input)    
    
    return tf.keras.Model(input, output, name=param["name"])


model = get_unet({"name":"unet",
                  "input_shape": (512,672), 
                  "n_class": 3,
                  "filter_multiplier": 5,
                  "n_depth": 1,
                  "kernel_initialization": "he_normal",
                  "dropout": 0.01,
                  "kernel_size": (8,8),
                  "upsample_channel_multiplier": 2})

In [ ]:
model.summary()

#### 2.3.1 Large (normal) Skyrmion U-Net

The normal, larger U-Net is defined further below in a section. Here is the architecture:

In [ ]:
plotfig(basis_dir+"notebook_figures/u_net_architecture_3.png",320)

It can also be created using the function `get_unet`, with larger parameters. For the normal (larger) U-Net, we would define:

```python
model = get_unet({
    "name": "unet",
    "input_shape": img_size,
    "n_class": 3,
    "filter_multiplier": 16,
    "n_depth": 4,
    "kernel_initialization": "he_normal",
    "dropout": 0.1,
    "kernel_size": (3, 3),
    "upsample_channel_multiplier": 8
})


## 3. Loss function
### 3.1. Loss function and training

AI models can be described as follows:

$$ \mathbf{z} = \mathbf{f}_{\vec{\theta}}(\mathbf{x}) $$

where $\vec{\theta}$ is a high-dimensional vector containing all parameters, $\mathbf{z}$ is the output tensor, and $\mathbf{x}$ is the input. Multiple function calls $ \mathbf{f}_{\vec{\theta}}(\mathbf{x}_i) $ for several data points $\mathbf{x}_i, i\in \{1,\ldots,n\}$ are written as $ \mathbf{f}_{\vec{\theta}}(\{x_i\}) $ in the following.

The loss function $ L(\mathbf{f}_{\vec{\theta}}(\{x_i\}), \{y_i\}) $ quantifies the error between the model's predictions $ \mathbf{f}_{\vec{\theta}}(\{x_i\}) $ and the true labels $ \{y_i\} $ corresponding to the data points $ \{x_i\} $. It measures how well the model's output matches the true labels, and minimizing this loss is the objective during training. If the error between the model's predictions and the true labels (groundtruth) is large, the loss function will also yield a large value, indicating poor model performance.

The training of a neural network aims to obtain the global minimum, i.e., it satisfies:

$$ \nabla_{\vec{\theta}} L(\mathbf{f}_{\vec{\theta}}(\{x_i\}), \{y_i\}) = 0 $$

To achieve this, the network must undergo training. The simplest approach is:

$$ \vec{\theta}_{i+1} = \vec{\theta}_{i} - \epsilon \left. \nabla_{\vec{\theta}_{i}} L(\mathbf{f}_{\vec{\theta}_{i}}(\{x_i\}), \{y_i\})  \right|_{\vec{\theta}_i}, $$

where $\epsilon$ is the learning rate.

However, in this session, for training the large skyrmion U-Net, we employed more advanced and superior training algorithms. The gradient $ \nabla_{\vec{\theta}} L $ can be calculated using backpropagation, which leverages the chain rule for derivatives.

The update of $ \vec{\theta} $ using the gradient of $ L(\mathbf{f}_{\vec{\theta}}(\{x_i\}), \{y_i\})  $ is typically done in such a way that $ \{x_i\} $ is not the entire dataset but rather a subset called a **batch**. Batches are used in machine learning to fit data into limited RAM and improve training efficiency while stabilizing gradient updates. This is not important for our mini example but becomes crucial with large datasets. When all the data points in the dataset (divided into batches) have been processed once, this is referred to as an **epoch**. Generally, the model is trained over several epochs to achieve optimal performance.

### 3.2. Simple Basic Example of Loss Functions

Verry simple example that demonstrates training, batches and epochs.

We now use this function (with fixed parameters $a,b,c,d$) to generate data points with Gaussian noise and learn from the data points the parameters $a$, $b$, $c$, and $d$:
$$f(x) = a \tanh(c x + d) + b$$

In [ ]:
n_data = 1000
xmin,xmax = -10,10
x =  np.linspace(xmin,xmax,n_data)
ap,bp,cp,dp = 0.4,1.5,0.6,1.6
y = ap*np.tanh(cp*x+dp)+bp
x += np.random.randn(n_data)*0.04
y += np.random.randn(n_data)*0.04
data = np.vstack((x,y)).T
fig,ax = plt.subplots(dpi=100)
ax.plot(data[:,0],data[:,1],"ro",ms=1,label="Data function")
ax.set_xlabel("x")
ax.set_ylabel("y")
xl = np.linspace(xmin,xmax)
ax.plot(xl,ap*np.tanh(cp*xl+dp)+bp,color="green",label="Original function",lw=2,zorder=9000)
ax.legend()

The trainings data look like:

In [ ]:
print(data)

Our model, which we aim to learn, is given by:

$$f(x)=a \tanh(c x + d) +b $$

where $ \theta = (a, b, c, d) $. Next, we define a loss function:

$$L(\{Y_i\},\{y_i\}) = \sum_i (Y_i-y_i)^2$$

where the summation runs over all the values in the batch.

The parameter update equation,

$$ \vec{\theta}_{i+1} = \vec{\theta}_{i} - \epsilon \left. \nabla_{\vec{\theta}} L(\mathbf{f}_{\vec{\theta}}(\{x_i\}), \{y_i\})  \right|_{\vec{\theta}_i}, $$
then becomes with $\Delta= a_i \tanh(c x_i + d_i) + b_i-y_i$ :

$$ a_{i+1} = a_{i} - \epsilon \frac{\partial}{\partial a_i}  \sum_i \Delta_i^2 = a_{i} - \epsilon   \sum_i 2 \tanh(c x_i + d_i) \Delta_i$$
$$ b_{i+1} = b_{i} - \epsilon \frac{\partial}{\partial b_i}  \sum_i \Delta_i^2 = b_{i} - \epsilon   \sum_i 2  \Delta_i $$
$$ c_{i+1} = c_{i} - \epsilon \frac{\partial}{\partial c_i}  \sum_i \Delta_i^2 = a_{i} - \epsilon \sum_i 2  a_i x_i (1-\tanh(c x_i + d_i)^2) \Delta_i $$
$$ d_{i+1} = d_{i} - \epsilon \frac{\partial}{\partial d_i}  \sum_i \Delta_i^2 = d_{i} - \epsilon   \sum_i 2 a_i  (1-\tanh(c x_i + d_i)^2) \Delta_i $$


where is the prediction error and the summation again runs over all the values in the batch.

Split the data into batches:

In [ ]:
n_batch = 100
#split the data in batches
batches = [data[i*n_batch:min((i+1)*n_batch,n_data)] for i in range(int(np.ceil(n_data/n_batch)))]
print(batches)

Now train the parameters

In [ ]:
n_epochs = 500
epsilon = 3e-3 #learning rate
a,b,c,d = 1,1,1,1# initial training parameters
training_history = []
for epoch in range(n_epochs):
    loss = 0
    for i in range(len(batches)):
        x_batch,y_batch = batches[i][:,0],batches[i][:,1]
        y_pred = a * np.tanh(c * x_batch + d) + b
        loss += np.sum((y_pred - y_batch)**2)
    training_history.append((a,b,c,d,loss))

    
    for i in range(len(batches)):
        x_batch,y_batch = batches[i][:,0],batches[i][:,1]
        y_pred = a * np.tanh(c * x_batch + d) + b
        error = y_pred - y_batch
        an = a - epsilon * np.sum(2 * error * np.tanh(c * x_batch + d))
        bn = b - epsilon * np.sum(2 * error)
        cn = c - epsilon * np.sum(2 * a * x_batch * (1 - np.tanh(c * x_batch + d)**2) * error)
        dn = d - epsilon * np.sum(2 * a * (1 - np.tanh(c * x_batch + d)**2) * error)
        a,b,c,d = an,bn,cn,dn
        
    if epoch%100==0:
        print(f"Epoch {epoch}: a={a:.3f},b={b:.3f},c={c:.3f},d={d:.3f}")
    
#calculate loss of last epoch
loss = 0
for i in range(len(batches)):
    x_batch,y_batch = batches[i][:,0],batches[i][:,1]
    y_pred = a * np.tanh(c * x_batch + d) + b
    loss += np.sum((y_pred - y_batch)**2)
training_history.append((a,b,c,d,loss))

training_history = np.array(training_history)

Let's take a look at the training history, starting with the evolution of the training parameters:

In [ ]:
fig,ax = plt.subplots(dpi=300,figsize=(9,4))
ax.set_title("Training History")
ax.set_ylabel(r"Training parameters")
ax.set_xlabel(r"Epoch")

ax.plot(np.arange(1,1+len(training_history)),training_history[:,0],label="a",color="#1f77b4")
ax.axhline(ap,label="a (Ground truth)",color="#1f77b4",ls="--",zorder=-900)
ax.plot(np.arange(1,1+len(training_history)),training_history[:,1],label="b",color='#ff7f0e')
ax.axhline(bp,label="b (Ground truth)",color="#ff7f0e",ls="--",zorder=-900)
ax.plot(np.arange(1,1+len(training_history)),training_history[:,2],label="c",color='#2ca02c')
ax.axhline(cp,label="c (Ground truth)",color="#2ca02c",ls="--",zorder=-900)
ax.plot(np.arange(1,1+len(training_history)),training_history[:,3],label="d",color='#d62728')
ax.axhline(dp,label="d (Ground truth)",color="#d62728",ls="--",zorder=-900)
ax.legend(ncols=2)
fig.tight_layout()

And now, let's look at the loss:

In [ ]:
fig,ax = plt.subplots(dpi=300,figsize=(9,4),ncols=2)
ax[0].set_title("Training History")
ax[0].set_ylabel(r"Loss $L$")
ax[0].set_xlabel(r"Epoch")
ax[0].plot(np.arange(1,1+len((training_history))),training_history[:,4])
ax[1].set_title("Training History, shifted Loss function")
ax[1].set_ylabel(r"Loss $L-L_0$")
ax[1].set_xlabel(r"Epoch")
ax[1].plot(np.arange(1,1+len((training_history))),training_history[:,4]-np.min(training_history[:,4]))
ax[1].set_yscale("log")
fig.tight_layout()

And this is how the final trained (fitted) function looks:

In [ ]:
xl = np.linspace(xmin,xmax)
fig,ax = plt.subplots(dpi=100)
ax.plot(data[:,0],data[:,1],"ro",ms=1)
ax.plot(data[:,0],data[:,1],"ro",ms=1,label="Data points")
ax.plot(xl,a*np.tanh(c*xl+d)+b,label="Trained function",lw=5)
ax.plot(xl,ap*np.tanh(cp*xl+dp)+bp,color="green",label="Original function",lw=2,zorder=9000)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()

### 3.3 Cross entropy

U-Net can be described as follows:

$$ \mathbf{z} = \mathbf{f}_{\vec{\theta}}(\mathbf{x}) $$

where $\vec{\theta}$ is a high-dimensional vector containing all parameters, $\mathbf{z}$ is a 3-dimensional output tensor, and $\mathbf{x}$ is the input Kerr microscopy image. In $\mathbf{f}$ are encapsulated all the operations such as convolution, max pooling, up-convolution, batch normalization, ... .

$\mathbf{z}$ is converted to probabilities using softmax:

$$ p_{(x,y),i} = \frac{\exp(\mathbf{z}_{(x,y),i})}{\sum_{i\in \mathrm{classes}} \exp(\mathbf{z}_{(x,y),i})} $$

The output mask $\mathbf{m}$ is then determined by:

$$ \mathbf{m}_{(x,y)} = \mathrm{arg\,max}_i\; p_{(x,y),i}$$

The cross-entropy loss (with averaging over all the pixels) is given by:

$$ L = - \frac{1}{\sum_{x,y} 1} \sum_{x,y} \sum_{i=1}^3 \;w_i\;(p_{\mathrm{ground-truth}})_{(x,y),i} \log(p_{(x,y),i}) $$

where $\sum_{(x,y)}$ in our case sums over several examples.

Let's define the cross-entropy function. This function returns a loss entropy value based on the weights assigned to the classes.

In [ ]:
def get_cross_entropy_loss(weight):
    weight = tf.convert_to_tensor(weight,dtype=np.float32)
    def loss(ytrue,ypred):
        p = tf.clip_by_value(ypred/(tf.math.reduce_sum(ypred,axis=-1,keepdims=True)),tf.keras.backend.epsilon(),1-tf.keras.backend.epsilon())
        return -tf.math.reduce_mean(tf.math.reduce_sum(weight*ytrue*tf.math.log(p),axis=-1))
    return loss

In [ ]:
get_cross_entropy_loss([6,1,1])

## 4. Dataset processing for training

Define a plot function to plot several images in a grid.

In [ ]:
def plot_results(l,ltitle=None,nrows=None,ncols=None,dpi=100,s0=1,suptitle=None):
    if nrows==None and ncols==None:
        nc = int(np.ceil(np.sqrt(len(l))))
        nr = int(np.ceil(len(l)/nc))
    if ncols!=None and ncols!=None:
        nc = ncols
        nr = nrows
    if nrows!=None and ncols==None:
        nr = nrows
        nc = int(np.ceil(len(l)/nr))
    if ncols!=None and nrows==None:
        nc = ncols
        nr = int(np.ceil(len(l)/nc))
    fig,ax = plt.subplots(nrows=nr,ncols=nc,dpi=dpi,figsize=(nc*s0,nr*s0))
    if suptitle != None:
        fig.suptitle(suptitle,fontsize=40,y=0.99)
    ax = ax.ravel()
    for i in range(len(ax)):
        ax[i].axis("off")
    
    for i in range(min(len(l),len(ax))):
        ax[i].imshow(l[i],cmap="gray")
        if ltitle!= None:
            ax[i].set_title(ltitle[i])
    fig.tight_layout()

### 4.1 Augmentation

Augmentation is used in model training to enhance the diversity of the training data, improve model generalization, and reduce overfitting. The augmentation is performed using the Albumentations package, which offers a wide range of powerful and efficient image transformation techniques. These techniques include random flipping, rotation, Gaussian noise, affine transformations (such as translation, scaling, and rotation), and adjustments to brightness and contrast — all of which simulate real-world variations in the data.

Define a data function that returns an augmentation function.

In [ ]:
def get_augmentation():
    return albumentations.Compose([
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.GaussNoise(p=1,std_range=(0,25/255)),
        albumentations.Affine(translate_percent=0.5,   scale=(0.85, 1.4),  rotate=(-90, 90),p=1,keep_ratio=True,border_mode=cv2.BORDER_REFLECT_101),
        albumentations.RandomBrightnessContrast(brightness_limit=0.3,contrast_limit=0.5,p=1)], p=1)
aug = get_augmentation()

Make an examples with this augmentation function:

In [ ]:
lexample_img = []
lexample_aug = []
for ix,subtable in dataset_table.groupby("source_id"):
    if ix not in [7, 8, 10, 11, 13, 23, 135]: continue
    img = (np.array(Image.open(subtable.iloc[0].img_fn)))  
    lexample_img.append(img)
    lexample_aug.append(aug(image=img)["image"])

plot_results(lexample_img,nrows=1,suptitle="Original images",s0=5,dpi=40)
plot_results(lexample_aug,nrows=1,suptitle="Augmented images",s0=5,dpi=40)

The augmented images may sometimes appear as if they don't originate from the original images. However, this is not the case; they are all results of augmentation.

For our training, we use the following data augmentation with slightly different parameters:

In [ ]:
def get_augmentation():
    return albumentations.Compose([
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.GaussNoise(p=1,std_range=(0,25/255)),
        albumentations.Affine(translate_percent=0.5,   scale=(0.85, 1.4),  rotate=(-90, 90),p=1,keep_ratio=True,border_mode=cv2.BORDER_REFLECT_101),
        albumentations.RandomBrightnessContrast(brightness_limit=0.25,contrast_limit=0.25,p=1)], p=1)

aug = get_augmentation()

And again some examples:

In [ ]:
lexample_img = []
lexample_aug = []
for ix,subtable in dataset_table.groupby("source_id"):
    if ix not in [7, 8, 10, 11, 13, 23, 135]: continue
    img = (np.array(Image.open(subtable.iloc[0].img_fn)))  
    lexample_img.append(img)
    lexample_aug.append(aug(image=img)["image"])

plot_results(lexample_img,nrows=1,suptitle="Original images",s0=5,dpi=40)
plot_results(lexample_aug,nrows=1,suptitle="Augmented images",s0=5,dpi=40)

### 4.2 Label smoothing

We apply label smoothing to the output mask to enhance training performance. This technique blends the original labels with a uniform distribution, preventing the model from relying too heavily on exact label values and enhancing generalization. The smoothed probability is defined as follows: 
$$p_{(x,y),i,\mathrm{training}} = (1-f) p_{(x,y),i,\mathrm{label}} + \frac{f}{\# \text{ classes}}.$$

In [ ]:
def label_smoothing(x,f,num_classes):
    return (1-f)*x+f/num_classes

fig,ax = plt.subplots(ncols=3,dpi=300,figsize=(12,2.9),constrained_layout=True)
im = ax[0].imshow(segmentation_mask_label_class,cmap="gray",vmin=0,vmax=2)
col = fig.colorbar(im,ax=ax[0])
ax[0].set_title("Segmentation mask"+"\n"+r"No label smoothing $f=0$")

im = ax[1].imshow(label_smoothing(segmentation_mask_label_class,0.1,3),cmap="gray",vmin=0,vmax=2)
col = fig.colorbar(im,ax=ax[1])
ax[1].set_title("Segmentation mask"+"\n"+r"Label smoothing $f=0.1$")

im = ax[2].imshow(label_smoothing(segmentation_mask_label_class,0.9,3),cmap="gray",vmin=0,vmax=2)
col = fig.colorbar(im,ax=ax[2])
ax[2].set_title("Segmentation mask"+"\n"+r"Label smoothing $f=0.9$")

### 4.3 Data generator

A data generator efficiently loads and feeds data in batches during training, ideal for large datasets. It supports real-time augmentation, image shuffling, and label smoothing.

The data generator also renormalizes the frames from RGB 8-bit values (0-255) to floating-point values in the range 0-1, as data normalization is always required for neural networks.

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Training'
    def __init__(self, images, labels, batch_size, n_class=3, smoothing=False,shuffle=True, aug=None):
        super().__init__()
        'Initialization'
        self.n_class = n_class
        if labels.shape[:3] != images.shape[:3]:
            raise Exception("Shape not fit")

        self.len_data = labels.shape[0]
        self.shape_data = labels.shape[1:3]
        self.labels = labels
        self.images = images

        self.batch_size = batch_size
        self.shuffle = shuffle
        self.smoothing = smoothing
        
        self.aug = aug
        self.on_epoch_end()

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len_data)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.len_data/ self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X,y = (self.images[indexes]).copy(), (self.labels[indexes]).copy()    
        if self.aug is not None:      
            self.data_augmentation(X, y)
        y = y.astype(dtype=np.float16)
        if self.smoothing:
            y[y==0] = self.smooth_labels(0.0,factor=0.2)
            y[y==1] = self.smooth_labels(1.0,factor=0.2)
        return X/255,y
        
    def smooth_labels(self, labels, factor=0.1):
        return labels*(1 - factor)+(factor / self.n_class)
    
    def data_augmentation(self, X, y):
        for i in range(self.batch_size):
            augmented = self.aug(image=X[i],mask=y[i])
            X[i] = augmented["image"]
            y[i] = augmented["mask"]

### 4.4 Splitting of Dataset into Training, Validation, and Test Set

For training, we split our data set into training, validation, and test sets. The training set is used to train the model by allowing it to learn the underlying patterns in the data. The validation set is used to tune hyperparameters and assess the model’s performance during training to prevent overfitting. Finally, the test set is employed to evaluate the model’s generalization ability on unseen data, providing a realistic measure of its performance in real-world scenarios.

In [ ]:
fig,ax = plt.subplots(figsize=(8, 1),dpi=200)
ax.text(0.2, 0.7, 'Training Set', ha='center', va='center', fontsize=12, bbox=dict(facecolor='#4CAF50',lw=0, alpha=0.6,pad=5))
ax.text(0.5, 0.7, 'Validation Set', ha='center', va='center', fontsize=12, bbox=dict(facecolor='#2196F3',lw=0, alpha=0.6,pad=5))
ax.text(0.85, 0.7, 'Test Set', ha='center', va='center', fontsize=12, bbox=dict(facecolor='#FF9800',lw=0, alpha=0.6,pad=5))
ax.text(0.35, 0.9,'Used during training process', ha='center')
ax.text(0.86, 0.9,'Used for model evaluation',ha='center')
ax.axis('off')
ax.set_title('Data Split for AI Model Training')
ax.set_ylim(0.6,1.1)

The same source ID indicates that the images/frames originate from the same video. Therefore, we split the train, test, and validation sets such that images/frames from a single video remain in the same set. Thus, the training, validation, and test sets are defined based on the source ID.

In [ ]:
train_ix = np.array([ix for ele in sorted(list(set([10,11,20,23,7,135]))) for ix in dataset_table[dataset_table.source_id==ele].index.to_numpy()])
test_ix = np.array([ix for ele in sorted(list(set([6,9]))) for ix in dataset_table[dataset_table.source_id==ele].index.to_numpy()])
val_ix = np.array([ix for ele in sorted(list(set([8,13]))) for ix in dataset_table[dataset_table.source_id==ele].index.to_numpy()])

print(train_ix)

Only the indices are stored from the data table, so you can then find the image from the data table. Later, all images will also be loaded in the same order as in the data table, and then you can access the images using the index.

Now an example of one image taken from each video used, along with the split into training, validation, and test sets.

In [ ]:
def group_after_source(lix,ncolmax=6,suptitle=None):
    table = dataset_table.iloc[lix]
    plot_results([np.array(Image.open(b.iloc[0].img_fn)) for a,b in table.groupby("source_id")],[str(a) for a,b in table.groupby("source_id")],suptitle=suptitle,ncols=6,dpi=100,s0=5)
group_after_source(train_ix,suptitle="Training set")
group_after_source(val_ix,suptitle="Validation set")
group_after_source(test_ix,suptitle="Test set")

### 4.5 Loading images and labels

Loading of images and labels. The loaded arrays/tensors from the function `load_img_label_data` have a shape of 
$$(\text{number of images}, \text{image height}, \text{image width}).$$

The index of the first dimension of these arrays corresponds to the index of the training, test, and validation sets.

In [ ]:
def load_img_label_data(lix):
    return np.array([Image.open(ele) for ele in dataset_table.iloc[lix].img_fn]), \
           np.array([trafo_rgb_to_class(np.array(Image.open(ele))) for ele in dataset_table.iloc[lix].label_fn])
    

train_img,train_label = load_img_label_data(train_ix)
val_img,val_label = load_img_label_data(val_ix)
test_img,test_label = load_img_label_data(test_ix)
img_size = test_img.shape[1:]

In [ ]:
print("train_img.shape=",train_img.shape)
print("train_img=\n",train_img)

print("train_label.shape=",train_label.shape)
print("train_label=\n",train_label)

## 5 Benchmarking 

In [ ]:
label_example = train_label[1]
fig,ax = plt.subplots(dpi=150,figsize=(6,3))
ax.imshow(trafo_class_to_rgb(label_example))
ax.set_title("Groundtruth")
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                    cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                    ax=ax, ticks=[0.5, 1.5, 2.5])
cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])
fig.tight_layout()

fig,ax = plt.subplots(ncols=3,dpi=300,figsize=(9,3))
ax[0].set_title("Perfect prediction")
ax[0].imshow(trafo_class_to_rgb(label_example))
ax[1].set_title("Wrong prediction\n wrong_label_1")
wrong_label_1 = np.random.randint(0,3,label_example.shape)
ax[1].imshow(trafo_class_to_rgb(wrong_label_1))
ax[2].set_title("Wrong prediction\n wrong_label_2")
wrong_label_2 = label_example.copy()
wrong_label_2[wrong_label_2==0] = 5
wrong_label_2[wrong_label_2==2] = 0
wrong_label_2[wrong_label_2==5] = 2
ax[2].imshow(trafo_class_to_rgb(wrong_label_2))

for i in range(3):
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                    cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                    ax=ax[i], ticks=[0.5, 1.5, 2.5],orientation='horizontal')
    cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])

fig.tight_layout()

To provide a quantitative description of the benchmarking process, we perform a pixel-wise evaluation and reduce it to a binary classification: 
- True class (true = skyrmion)
- False class (false = defect or background).

Using this classification, we can construct the confusion matrix.

|                | Actual Positive | Actual Negative |
|----------------|-----------------|------------------|
| **Predicted Positive** | **TP** (True Positive) | **FP** (False Positive) |
| **Predicted Negative** | **FN** (False Negative) | **TN** (True Negative) |


With this, we can define the **Matthews Correlation Coefficient (MCC)**:

$$ \mathrm{MCC} = \frac{\mathrm{TP} \cdot \mathrm{TN} - \mathrm{FP} \cdot \mathrm{FN}}{\sqrt{(\mathrm{TP} + \mathrm{FP})(\mathrm{FN} + \mathrm{TN})(\mathrm{TP} + \mathrm{FN})(\mathrm{FP} + \mathrm{TN})}} $$

where:
- $\mathrm{TP}$ = number of true positives  
- $\mathrm{TN}$ = number of true negatives  
- $\mathrm{FP}$ = number of false positives  
- $\mathrm{FN}$ = number of false negatives  


So let's implement the MCC in code now:

In [ ]:
def get_TF_PN(y_true,y_pred,ix0):
    m1,m2 = y_true==ix0,y_pred==ix0
    im1,im2 = tf.math.logical_not(m1),tf.math.logical_not(m2)
    TP = tf.math.reduce_mean(tf.cast(tf.math.logical_and(m1,m2),dtype=np.float64))
    TN = tf.math.reduce_mean(tf.cast(tf.math.logical_and(im1,im2),dtype=np.float64))
    FP = tf.math.reduce_mean(tf.cast(tf.math.logical_and(im1,m2),dtype=np.float64))
    FN = tf.math.reduce_mean(tf.cast(tf.math.logical_and(m1,im2),dtype=np.float64))
    return TP,TN,FP,FN

def get_mcc_from_TF_PN(TP,TN,FP,FN):
    denom = tf.keras.ops.sqrt((TP + FN) * (FP + TN) * (FP + TP) * (FN + TN))
    val = (TP * TN - FP * FN) / denom
    return  tf.where(tf.equal(denom, 0), tf.constant(0, dtype=tf.float64), val)

get_mcc = lambda x,y,ix0: get_mcc_from_TF_PN(*get_TF_PN(x,y,ix0)).numpy()

For the example above, the MCC is now:

In [ ]:
fig,ax = plt.subplots(ncols=3,dpi=300,figsize=(9,3))

mcc_1 = get_mcc(label_example,label_example,0)
mcc_2 = get_mcc(label_example,wrong_label_1,0)
mcc_3 = get_mcc(label_example,wrong_label_2,0)

print("MCC label_example:",mcc_1)
print("MCC wrong_label_1:",mcc_2)
print("MCC wrong_label_2:",mcc_3)

ax[0].set_title(f"MCC={mcc_1:.3f}"+"\n"+"Perfect prediction\n")
ax[0].imshow(trafo_class_to_rgb(label_example))
ax[1].set_title(f"MCC={mcc_2:.3f}"+"\n"+"Wrong prediction\n wrong_label_1")
ax[1].imshow(trafo_class_to_rgb(wrong_label_1))
ax[2].set_title(f"MCC={mcc_3:.3f}"+"\n"+"Wrong prediction\n wrong_label_2")
ax[2].imshow(trafo_class_to_rgb(wrong_label_2))

for i in range(3):
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                    cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                    ax=ax[i], ticks=[0.5, 1.5, 2.5],orientation='horizontal')
    cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])
    
fig.tight_layout()

Additionally, we need the following code for the MCC calculation during the training process:

In [ ]:
class MCC(tf.keras.metrics.Metric):
    def __init__(self, ix0, name="MCC", **kwargs):
        super().__init__(name=name,dtype=tf.float64,**kwargs)
        self.ix0 = ix0
        self.tp = self.add_variable(shape=(),name="TP", initializer="zeros",dtype=tf.float64)
        self.tn = self.add_variable(shape=(),name="TN", initializer="zeros",dtype=tf.float64)
        self.fp = self.add_variable(shape=(),name="FP", initializer="zeros",dtype=tf.float64)
        self.fn = self.add_variable(shape=(),name="FN", initializer="zeros",dtype=tf.float64)
        self.tot = self.add_variable(shape=(),name="TOT", initializer="zeros",dtype=tf.int64)
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        TP,TN,FP,FN = get_TF_PN(tf.argmax(y_true,axis=-1),tf.argmax(y_pred,axis=-1),self.ix0)
        totn1 = tf.cast(self.tot+1,tf.float64)
        self.tp.assign_add((TP-self.tp)/totn1)
        self.tn.assign_add((TN-self.tn)/totn1)
        self.fp.assign_add((FP-self.fp)/totn1)
        self.fn.assign_add((FN-self.fn)/totn1)
        self.tot.assign_add(1)
        
    def reset_state(self):
        self.tp.assign(0)
        self.tn.assign(0)
        self.fp.assign(0)
        self.fn.assign(0)
        self.tot.assign(0)    
    
    def result(self):
        return get_mcc_from_TF_PN(self.tp,self.tn,self.fp,self.fn)

## 6. Training

### 6.1 Initialization and Performing the Training

Define where the model is stored, initialize the training process, especially the models and the data generator.

For training, we use the **Adam optimizer** for training with cross-entropy loss. Additionally, we monitor the training process using the Matthews correlation coefficient (MCC). Furthermore, we define the data generators for the training and validation sets, and we set the batch size to 5.

In [ ]:
modeln = "model_small"
folder = basis_dir+"training/"
modelfn = folder+modeln+".keras"

if not os.path.exists(folder):
    os.makedirs(folder)

model = get_unet({"name":"unet","input_shape": (512,672), "n_class":3,"filter_multiplier":5,"n_depth":1,
                  "kernel_initialization":"he_normal","dropout":0.01,"kernel_size":(8,8),"upsample_channel_multiplier":2})

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),loss=get_cross_entropy_loss([6,1,1]),metrics=["accuracy",MCC(0)])

batch_size = 5
basis = np.eye(3,dtype=np.uint8)
dg_train = DataGenerator(train_img,basis[train_label],batch_size,smoothing=False,aug=get_augmentation())
dg_val = DataGenerator(val_img,basis[val_label],batch_size,smoothing=False,aug=get_augmentation())

Now we perform the training. We train for 15 epochs, passing the data and the validation training set. Furthermore, we reduce the learning rate if the validation loss does not decrease after a certain number of epochs. Each time the validation loss reaches a new minimum, we save the model. Finally, we save the training history.

If the training does not work, a pre-trained small model along with its training history is already saved, which you can load afterwards in case the training doesn't work.

In [ ]:
history = model.fit(x=dg_train,validation_data=dg_val,epochs=15,verbose=1,batch_size=batch_size,callbacks=[
tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min',factor=0.5, patience=2, min_lr=1e-6, verbose=1),
tf.keras.callbacks.ModelCheckpoint(modelfn, verbose=1,monitor='val_loss',mode='min',save_best_only=True,save_weights_only=False)])

hist_df = pd.DataFrame(history.history)  
hist_df.to_csv(folder+modeln+"_history.csv")

In case the **training fails**, then **execute this cell** to load a training curve and a trained U-Net (with the same architecture as discussed above) from a previous training.

In [ ]:
try:
    hist_df
except:
    modeln = "model_small"
    folder = basis_dir+"training_result/"
    modelfn = folder+modeln+".keras"
    hist_df = pd.read_csv(folder+modeln+"_history.csv")
    mini_unet_loaded_from_repository = True

### 6.2 Training history

Plot the Training Performance History

In [ ]:
fig,ax = plt.subplots(ncols=3,figsize=(8,3),dpi=300)
fig.suptitle("Training history - Validation Set",fontsize=14)
ax[0].plot(np.arange(1,len(hist_df)+1),hist_df["val_loss"],color="indianred",lw=4)
ax[1].plot(np.arange(1,len(hist_df)+1),hist_df["val_accuracy"],color="indianred",lw=4)
ax[2].plot(np.arange(1,len(hist_df)+1),hist_df["val_MCC"],color="indianred",lw=4)
for i in range(3):
    ax[i].set_xlabel("Epoch")
    ax[i].set_xticks(np.arange(0,len(hist_df)+1,5))
    
ax[0].set_ylabel("Cross-entropy loss")
ax[0].set_title("Cross-entropy loss")
ax[1].set_ylabel("Accuracy")
ax[1].set_title("Accuracy")
ax[2].set_ylabel("MCC")
ax[2].set_title("MCC")
fig.tight_layout()

fig,ax = plt.subplots(ncols=3,figsize=(8,3),dpi=300)
fig.suptitle("Training history - Trainings Set",fontsize=14)
ax[0].plot(np.arange(1,len(hist_df)+1),hist_df["loss"],color="green",lw=4)
ax[1].plot(np.arange(1,len(hist_df)+1),hist_df["accuracy"],color="green",lw=4)
ax[2].plot(np.arange(1,len(hist_df)+1),hist_df["MCC"],color="green",lw=4)
for i in range(3):
    ax[i].set_xlabel("Epoch")
    ax[i].set_xticks(np.arange(0,len(hist_df)+1,5))
ax[0].set_ylabel("Cross-entropy loss")
ax[0].set_title("Cross-entropy loss")
ax[1].set_ylabel("Accuracy")
ax[1].set_title("Accuracy")
ax[2].set_ylabel("MCC")
ax[2].set_title("MCC")
fig.tight_layout()

hist_df

## 7. Prediction & Benchmarking with the trained model


### 7.1 Prediction

Let's make a prediction on an image of the test set.

Pick one image out of the test set and normalize the values to the range

In [ ]:
img_to_be_predicted = test_img[0]/255
img_to_be_predicted

In [ ]:
fig,ax = plt.subplots()
ax.imshow(img_to_be_predicted,cmap="gray")

The filename of the trained model is:

In [ ]:
modelfn

Load the trained model:

(If the training fails and the pre-trained model Mini Skyrmion U-Net is loaded from the repository and executed on the CPU, the model must be redefined because the pre-trained Skyrmion U-Nets were trained on the GPU with float_16)

In [ ]:
model = tf.keras.models.load_model(modelfn,compile=False,custom_objects={'MishLayer': MishLayer})

if (not gpu_available()) and mini_unet_loaded_from_repository:
    #create identical model, only with pure float_32 policy
    nmodel = get_unet({"name":"unet","input_shape": (512,672), "n_class": 3,
              "filter_multiplier": 5,"n_depth": 1,"kernel_initialization": "he_normal",
              "dropout": 0.01,"kernel_size": (8,8),"upsample_channel_multiplier": 2})
    nmodel.set_weights(model.weights)
    model = nmodel

Now make the Prediction with the U-Net. An additional dimension is added around the data since the U-Net input expects batches of images. Since the final layer outputs a probability for each class $i$, an argmax operation must be performed afterward: $ \mathbf{m}_{(x,y)} = \mathrm{arg\,max}_i\; p_{(x,y),i}$

In [ ]:
p = model(np.array([img_to_be_predicted]))[0]
p

In [ ]:
segmented_mask = np.argmax(p,axis=-1)
segmented_mask

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=2,dpi=300,figsize=(6,6),constrained_layout=True)
ax = ax.flatten()
ax[0].imshow(img_to_be_predicted,cmap="gray")
im = ax[1].imshow(segmented_mask,cmap="gray")
col = fig.colorbar(im,ax=ax[1])
col.set_ticks([0,1,2])
ax[2].imshow(trafo_class_to_rgb(segmented_mask))
ax[3].imshow(trafo_class_to_rgb(test_label[0]))

ax[0].set_title("Kerr micrograph")
ax[2].set_title("Predicted Mask\n(RGB code for the 3 classes)")
ax[1].set_title("Predicted Mask\n(Class index for the 3 classes)")
ax[3].set_title("Groundtruth Mask\n(RGB code for the 3 classes)")

for i in [2,3]:
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                    cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                    ax=ax[i], ticks=[0.5, 1.5, 2.5],orientation='horizontal')
    cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])

Now a prediction function that also predicts in batches (with the batches processed in **parallel on GPU**), allowing a large number of images to be predicted efficiently. The principle remains the same:

In [ ]:
def predict(x,modelfn,batch_size=5,normalize_255=True):
    model = tf.keras.models.load_model(modelfn,compile=False,custom_objects={'MishLayer': MishLayer})

    if not gpu_available():
        #create identical model, only with pure float_32 policy
        batch_size = 1
        nmodel = get_unet({"name":"unet","input_shape": (512,672), "n_class": 3,
                  "filter_multiplier": 5,"n_depth": 1,"kernel_initialization": "he_normal",
                  "dropout": 0.01,"kernel_size": (8,8),"upsample_channel_multiplier": 2})
        nmodel.set_weights(model.weights)
        model = nmodel
    
    n = int(np.ceil(len(x)/batch_size))
    lix = [np.array(range(j*batch_size,min((j+1)*batch_size,len(x)))) for j in range(n)]
    ylabel = np.zeros(x.shape,dtype=np.uint8)
    progbar = tf.keras.utils.Progbar(n)
    for i in range(n):            
        progbar.update(i)
        input = x[lix[i]]
        if normalize_255:
            input = input/255
        ylabel[lix[i]] = model.predict(input,verbose=False).argmax(-1)
    progbar.update(n,finalize=True)
    return ylabel

And now, prediction of images in the training, validation, and test sets:

In [ ]:
total_ix = np.hstack((test_ix,val_ix,train_ix))
total_img = np.vstack((test_img,val_img,train_img))
total_label = np.vstack((test_label,val_label,train_label))

S = set(np.hstack((test_ix,val_ix,train_ix)))&set([b.iloc[0].name for a,b in dataset_table.groupby("source_id")])
total_pred = predict(total_img,modelfn,5)
for ix0 in range(len(total_img)):
    if total_ix[ix0] not in S: continue
    fig,ax = plt.subplots(ncols=3,figsize=(8,3),dpi=200)
    for e in range(3):
        ax[e].axis("off")
    ax[0].imshow(total_img[ix0],cmap="gray")
    ax[1].imshow(trafo_class_to_rgb(total_label[ix0]))
    ax[2].imshow(trafo_class_to_rgb(total_pred[ix0]))
    ax[0].set_title("Kerr image")
    ax[1].set_title("Ground truth")
    ax[2].set_title("Predicted label")
    if total_ix[ix0] in train_ix:
        fig.suptitle("Image used in training",y=1.02,fontsize=16)
    elif total_ix[ix0] in test_ix:
        fig.suptitle("Image used in testing",y=1.02,fontsize=16)
    elif total_ix[ix0] in val_ix:
        fig.suptitle("Image used in validation",y=1.02,fontsize=16)
    fig.tight_layout()

    for i in range(3):
        cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                        cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                        ax=ax[i], ticks=[0.5, 1.5, 2.5],orientation='horizontal')
        cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])
        if i == 0:
            cbar.ax.set_visible(False)

### 7.2 Benchmarking with MCC

Benchmark with the MCC now the complete dataset by performing a benchmark on all images (validation, test, and train set) and once on the images of the test set:

In [ ]:
def batch_mcc(x,label,modelfn,ix0=0,batch_size=5,normalize_255=True):
    model = tf.keras.models.load_model(modelfn,compile=False,custom_objects={'MishLayer': MishLayer})
    n = int(np.ceil(len(x)/batch_size))
    lix = [np.array(range(j*batch_size,min((j+1)*batch_size,len(x)))) for j in range(n)]
    L = []
    progbar = tf.keras.utils.Progbar(n)
    for i in range(n):            
        progbar.update(i)
        input = x[lix[i]]
        if normalize_255:
            input = input/255
        output = model.predict(input,verbose=False).argmax(-1)
        TP,TN,FP,FN = get_TF_PN(label[lix[i]],output,ix0)
        L.append({"n_size":tf.reduce_prod(tf.cast(output.shape,dtype=tf.int64)),"TP":TP,"TN":TN,"FP":FP,"FN":FN})
    progbar.update(n,finalize=True)
    tot_n = sum([ele["n_size"] for ele in L])
    TP,TN,FP,FN = tf.Variable(0.0,dtype=tf.float64),tf.Variable(0.0,dtype=tf.float64),tf.Variable(0.0,dtype=tf.float64),tf.Variable(0.0,dtype=tf.float64)
    for ele in L:
        w = ele["n_size"]/tot_n
        TP.assign_add(w*ele["TP"])
        TN.assign_add(w*ele["TN"])
        FP.assign_add(w*ele["FP"])
        FN.assign_add(w*ele["FN"])
    return get_mcc_from_TF_PN(TP,TN,FP,FN).numpy()

Now calculate the values on complete set and test set

In [ ]:
print(f"Pixelwise Matthews correlation coefficient on test set (true=skyrmion,false={{defect,background}}) \n= {batch_mcc(val_img,val_label,modelfn,0,batch_size):.3f}")
print(f"Pixelwise Matthews correlation coefficient  on complete set (true=skyrmion,false={{defect,background}}) \n= {batch_mcc(total_img,total_label,modelfn,0,batch_size):.3f}")

## 8. Training a large Skyrmion U-Net training tutorial with large dataset

This section demonstrates how to train a large U-Net model using a large dataset. **You do not need to train the following remaining cells in the notebook if you do not want to train the large U-net.**

A powerful GPU is required to train the large model; however, the code and concept are the same as what we have done up to this point.

In [ ]:
plotfig(basis_dir+"notebook_figures/u_net_architecture_3.png",320)

### 8.1 Download of Zenodo skyrmion U-Net repository

Comment this out only if you are going to train the large U-Net. Approximately 1GB of data will be downloaded for this purpose. The Zenodo repository by Winkler et al. (2024) [https://zenodo.org/records/10997175](https://zenodo.org/records/10997175) will be downloaded.

In [ ]:
"""
import wget
import zipfile
import glob

zenodo_folder = basis_dir+"zenodo_dataset/"
if not os.path.exists(zenodo_folder):
    os.makedirs(zenodo_folder)

    wget.download("https://zenodo.org/records/10997175/files/public_unet_skyrmion_dataset.zip",zenodo_folder+"zenodo_unet_skyrmion_dataset.zip")
    with zipfile.ZipFile(zenodo_folder+"zenodo_unet_skyrmion_dataset.zip","r") as zip:
        zip.extractall(zenodo_folder)
"""

### 8.2 Load dataset and split dataset

Load dataset

In [ ]:
zenodo_dataset_folder = basis_dir+"zenodo_dataset/public_unet_skyrmion_dataset/"
zenodo_dataset = pd.read_csv(zenodo_dataset_folder+"table.csv",sep=";")

In [ ]:
lfnimg = sorted(list(glob.iglob(zenodo_dataset_folder+"images/*.png")))
lfnlabel = sorted(list(glob.iglob(zenodo_dataset_folder+"labels/*.png")))
limg = np.array([np.array(Image.open(ele)) for ele in lfnimg])
llabel = np.array([trafo_rgb_to_class(np.array(Image.open(ele))) for ele in lfnlabel])
img_size = limg.shape[1:]

Split dataset in training, validation and test set

In [ ]:
with open(zenodo_dataset_folder+"partition.txt","r") as f:
    S = f.read()
traintest_batch = []
trainonly_batch = []
for ele in S.split("\n")[:-1]:
    lt = ele.split(";")
    if lt[0]=="train_test_val":
        traintest_batch.append([int(ele) for ele in lt[1:]])
    elif lt[0]=="only_training":
        trainonly_batch.append([int(ele) for ele in lt[1:]])
    else:
        print("error")


#Association of batches to training, validation and testing
train_batch,val_batch,test_batch = [0,1,4,5],[2],[3]
#Assignment of indices for each training object (image and label) to validation, test, and training
train_sources = [ele1 for ele in trainonly_batch for ele1 in ele]+[ele for ix in train_batch for ele in traintest_batch[ix]]
train_ix = np.array(sorted([ele1 for ele in train_sources for ele1 in zenodo_dataset[zenodo_dataset["source_id"]==ele].index]))
val_sources = [ele for ix in val_batch for ele in traintest_batch[ix]]
val_ix = np.array(sorted([ele1 for ele in val_sources for ele1 in zenodo_dataset[zenodo_dataset["source_id"]==ele].index]))
test_sources = [ele for ix in test_batch for ele in traintest_batch[ix]]
test_ix = np.array(sorted([ele1 for ele in test_sources for ele1 in zenodo_dataset[zenodo_dataset["source_id"]==ele].index]))

### 8.3 Augmentation & Data Generator

Now define the Augmentation and Data Generator

In [ ]:
#Batch size refers to how many images will be passed through the U-Net at the same time during training
#One must adjust the batch_size based on available RAM/VRAM.
batch_size = 5
    
#stronger augmentation, compared to the mini U-Net

def get_augmentation():
    return albumentations.Compose([
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.GaussNoise(p=1,std_range=(0,25/255)),
        albumentations.Affine(translate_percent=0.5,   scale=(0.85, 1.4),  rotate=(-90, 90),p=1,keep_ratio=True,border_mode=cv2.BORDER_REFLECT_101),
        albumentations.RandomBrightnessContrast(brightness_limit=0.3,contrast_limit=0.5,p=1)], p=1)


basis = np.eye(3,dtype=np.uint8)
#creation of data generators for train and test
dg_train = DataGenerator(limg[train_ix],basis[llabel[train_ix]],batch_size,smoothing=False,aug=get_augmentation())
dg_val = DataGenerator(limg[val_ix],basis[llabel[val_ix]],batch_size,smoothing=False,aug=get_augmentation())

### 8.4 Defining the large U-Net architecture

Define the large U-Net architecture based on the architecture plotted at the beginning of this Section.

In [ ]:
modeln = "model_large"
folder = basis_dir+"training/"

if not os.path.exists(folder):
    os.makedirs(folder)

modelfn = folder+modeln+".keras"

model = get_unet({"name":"unet","input_shape": img_size, "n_class":3,"filter_multiplier":16,"n_depth":4,
                  "kernel_initialization":"he_normal","dropout":0.1,"kernel_size":(3,3),"upsample_channel_multiplier":8})

In [ ]:
model.summary()

### 8.5 Training

Now we train the last model:

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=get_cross_entropy_loss([6.2,10,1.2]),metrics=["accuracy",MCC(0)])
history = model.fit(x=dg_train,validation_data=dg_val,epochs=15,verbose=1,batch_size=batch_size,callbacks=[
tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min',factor=0.5, patience=3, min_lr=1e-6, verbose=1),
tf.keras.callbacks.ModelCheckpoint(modelfn, verbose=1,monitor='val_loss',mode='min',save_best_only=True,save_weights_only=False)])

hist_df = pd.DataFrame(history.history)  
hist_df.to_csv(folder+modeln+"_history.csv")

Example plot of a prediction from the test set

In [ ]:
fig,ax = plt.subplots(ncols=3,figsize=(10,4),dpi=300)
ix0 = 800
ax[0].imshow(limg[ix0],cmap="gray")
ax[1].imshow(trafo_class_to_rgb(llabel[ix0]))
ax[2].imshow(trafo_class_to_rgb(predict(np.array([limg[ix0]]),modelfn,5)[0]))
ax[0].set_title("Kerr image")
ax[1].set_title("Ground truth")
ax[2].set_title("Predicted label")

for i in [0,1,2]:
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=matplotlib.colors.BoundaryNorm([0,1,2,3],3), 
                        cmap=matplotlib.colors.ListedColormap([(0,0,1),(0,1,0),(1,0,0)])),
                        ax=ax[i], ticks=[0.5, 1.5, 2.5],orientation='horizontal')
    cbar.set_ticklabels(['Background', 'Defects', 'Skyrmions'])
    if i == 0:
        cbar.ax.set_visible(False)

fig.tight_layout()

Benchmark now the large Skyrmion U-Net

In [ ]:
print(f"Pixelwise Matthews correlation coefficient on test set (true=skyrmion,false={{defect,background}}) \n= {batch_mcc(limg[test_ix],llabel[test_ix],modelfn,0,batch_size):.3f}")
print(f"Pixelwise Matthews correlation coefficient  on complete set (true=skyrmion,false={{defect,background}}) \n= {batch_mcc(limg,llabel,modelfn,0,batch_size):.3f}")

## 9. MDAI (Magnetic Data Aggregator for AI)

How can one obtain data from magnetic systems to train their own AI model? → Magnetic Data Aggregator for AI (MDAI) later in the session.


The discussed MDAI — Magnetic Data Aggregator for AI — repository can be found at: [https://github.com/kfjml/MDAI](https://github.com/kfjml/MDAI)

## 10. Additional information

Further information on the Skyrmion U-Net can be found in the paper: Labrie-Boulay et al., *Phys. Rev. Applied* **21**, 014014 (2023). The complete training data and models (the models are also included here in this repository) can be found in the Zenodo repository by Winkler et al. [https://zenodo.org/records/10997175](https://zenodo.org/records/10997175) (2024), which is used for training the large Skyrmion-Net in Section 6.